In [ ]:
import pandas as pd
import csv
import re
from bs4 import BeautifulSoup
import marqo
from tqdm import tqdm

In [ ]:
df = pd.read_csv('./data/full_drop.csv')
model = "hf/multilingual-e5-large"
url = "http://localhost:8882"
mq = marqo.Client(url=url)

In [ ]:
nan_count = df.isna().sum()

print(nan_count)

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(nan_count)

In [ ]:
df.shape

In [ ]:
df = pd.read_csv('./data/full_one_hot_encoded.csv')

def clean_text(txt):
    if pd.isna(txt):
        clean_text = ""
    else:   
        soup = BeautifulSoup(str(txt).strip(), 'html.parser')
        clean_text = soup.get_text()
        clean_text = re.sub('[^a-zA-Z0-9\s]', '', clean_text)
        clean_text = clean_text.lower()
    return clean_text

df['facts_clean'] = df['facts'].apply(clean_text)
df['issue_area'] = df['issue_area'].apply(clean_text)
df['legal_question'] = df['legal_question'].apply(clean_text)
df['conclusion'] = df['conclusion'].apply(clean_text)

df['sim1_facts_score'] = None
df['sim1_issue_area_score'] = None
df['sim1_legal_question_score'] = None
df['sim1_conclusion_score'] = None

df['sim2_facts_score'] = None
df['sim2_issue_area_score'] = None

df['sim1_facts_href'] = None
df['sim2_facts_href'] = None

df['sim1_issue_area_href'] = None
df['sim2_issue_area_href'] = None

df['sim1_legal_question_href'] = None

df['sim1_conclusion_href'] = None


In [ ]:
(df['facts'] == '').sum()

In [ ]:
nan_count = df.isna().sum()

print(nan_count)

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(nan_count)

In [ ]:
# df[df['href']==search_href]['sim3_facts_score']

In [ ]:
# df[[df['href']==search_href]]

In [ ]:
df.loc[df['href']=='https://api.oyez.org/cases/1984/83-1919']

In [ ]:
for index, row in tqdm(df.iterrows()):
    search_href = row['href']
    # facts = row['facts_clean']
    # results = mq.index("whole").search(facts, search_method='TENSOR')
    # results_df = pd.DataFrame(results['hits'])
    # same_doc_index = results_df[results_df['href']==search_href].index
    # results_df.drop(same_doc_index , inplace=True)
    # results_df[0:3]

    facts = row['facts_clean']
    print("===============")
    print(index)
    print(facts)
    print(search_href)
    print("===============")
    results = mq.index("whole_legal").search(facts, search_method='TENSOR')
    results_df = pd.DataFrame(results['hits'])
    same_doc_index = results_df[results_df['href']==search_href].index
    results_df.drop(same_doc_index , inplace=True)
    results_df = results_df[0:3]
    print(results_df[['href', '_score']])
    df.loc[df['href']==search_href, 'sim1_facts_score'] = results_df['_score'].iloc[0]
    df.loc[df['href']==search_href, 'sim2_facts_score'] = results_df['_score'].iloc[1]

    df.loc[df['href']==search_href, 'sim1_facts_href'] = results_df['href'].iloc[0]
    df.loc[df['href']==search_href, 'sim2_facts_href'] = results_df['href'].iloc[1]
    print("===============")
    print(results_df['_score'].iloc[0])
    print(results_df['_score'].iloc[1])
    print(results_df['_score'].iloc[2])
    print("***********************")
    print(df[df['href']==search_href]['sim1_facts_score'])
    print(df[df['href']==search_href]['sim2_facts_score'])

    print(df[df['href']==search_href]['sim1_facts_href'])
    print(df[df['href']==search_href]['sim2_facts_href'])
    print("===============")
    print("===============")
    print("===============")

    issue_area = row['issue_area']
    print("===============")
    print(index)
    print(issue_area)
    print(search_href)
    print("===============")
    results = mq.index("whole_legal").search(issue_area, search_method='TENSOR')
    results_df = pd.DataFrame(results['hits'])
    same_doc_index = results_df[results_df['href']==search_href].index
    results_df.drop(same_doc_index , inplace=True)
    results_df = results_df[0:3]
    print(results_df[['href', '_score']])
    df.loc[df['href']==search_href, 'sim1_issue_area_score'] = results_df['_score'].iloc[0]
    df.loc[df['href']==search_href, 'sim2_issue_area_score'] = results_df['_score'].iloc[1]

    df.loc[df['href']==search_href, 'sim1_issue_area_href'] = results_df['href'].iloc[0]
    df.loc[df['href']==search_href, 'sim2_issue_area_href'] = results_df['href'].iloc[1]
    print("===============")
    print(results_df['_score'].iloc[0])
    print(results_df['_score'].iloc[1])
    print("***********************")
    print(df[df['href']==search_href]['sim1_issue_area_score'])
    print(df[df['href']==search_href]['sim2_issue_area_score'])

    print(df[df['href']==search_href]['sim1_issue_area_href'])
    print(df[df['href']==search_href]['sim2_issue_area_href'])
    print("===============")
    print("===============")
    print("===============")

    legal_question = row['legal_question']
    print("===============")
    print(index)
    print(legal_question)
    print(search_href)
    print("===============")
    results = mq.index("whole_legal").search(legal_question, search_method='TENSOR')
    results_df = pd.DataFrame(results['hits'])
    same_doc_index = results_df[results_df['href']==search_href].index
    results_df.drop(same_doc_index , inplace=True)
    results_df = results_df[0:3]
    print(results_df[['href', '_score']])
    df.loc[df['href']==search_href, 'sim1_legal_question_score'] = results_df['_score'].iloc[0]

    df.loc[df['href']==search_href, 'sim1_legal_question_href'] = results_df['href'].iloc[0]
    print("===============")
    print(results_df['_score'].iloc[0])
    print("***********************")
    print(df[df['href']==search_href]['sim1_legal_question_score'])

    print(df[df['href']==search_href]['sim1_legal_question_href'])
    print("===============")
    print("===============")
    print("===============")

    conclusion = row['conclusion']
    print("===============")
    print(index)
    print(conclusion)
    print(search_href)
    print("===============")
    results = mq.index("whole_legal").search(conclusion, search_method='TENSOR')
    results_df = pd.DataFrame(results['hits'])
    same_doc_index = results_df[results_df['href']==search_href].index
    results_df.drop(same_doc_index , inplace=True)
    results_df = results_df[0:3]
    print(results_df[['href', '_score']])
    df.loc[df['href']==search_href, 'sim1_conclusion_score'] = results_df['_score'].iloc[0]

    df.loc[df['href']==search_href, 'sim1_conclusion_href'] = results_df['href'].iloc[0]
    print("===============")
    print(results_df['_score'].iloc[0])
    print("***********************")
    print(df[df['href']==search_href]['sim1_conclusion_score'])

    print(df[df['href']==search_href]['sim1_conclusion_href'])
    print("===============")
    print("===============")
    print("===============")

    # for index, row in results_df[0:3].iterrows():
    #     document_id = row['_id']
    #     doc = mq.index("whole").get_document(
    #         document_id=document_id,
    #         expose_facets=True
    #     )
                
        # facets = doc['_tensor_facets']
        # for i in range(len(doc['_tensor_facets'])):
        #     field = list(facets[i].keys())[0]
        #     embedding = facets[i]['_embedding']
        #     print(field)

        # df.loc[df['href']==row['href'], f'sim{index}_{field}_vector'] = str(embedding)

In [ ]:
nan_count = df.isna().sum()

print(nan_count)

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(nan_count)

In [ ]:
# for index, row in tqdm(df.iterrows()):
#     search_href = row['href']
#     facts = row['facts_clean']
#     results = mq.index("sentences").search(facts, search_method='TENSOR')
#     results_df = pd.DataFrame(results['hits'])
#     same_doc_index = results_df[results_df['href']==search_href].index
#     results_df.drop(same_doc_index , inplace=True)
#     results_df[0:3]

#     df.loc[df['href']==search_href, 'sim1_score_sent']=results_df['_score'].iloc[0]
#     df.loc[df['href']==search_href, 'sim1_href_sent'] = results_df['href'].iloc[0]
#     df.loc[df['href']==search_href, 'sim2_score_sent'] = results_df['_score'].iloc[1]
#     df.loc[df['href']==search_href, 'sim2_href_sent'] = results_df['href'].iloc[1]
#     df.loc[df['href']==search_href, 'sim3_score_sent'] = results_df['_score'].iloc[2]
#     df.loc[df['href']==search_href, 'sim3_href_sent'] = results_df['href'].iloc[2]

#     for index, row in results_df[0:3].iterrows():
#         document_id = row['_id']
#         facets = mq.index("full_orig_model_filevine_docs_sentences").get_document(
#             document_id=document_id,
#             expose_facets=True
#         )

#         df.loc[df['href']==row['href'], f'sim{index}_vector_sent'] = str(facets['_tensor_facets'][0]['_embedding'])

In [ ]:
df.to_csv('./data/final.csv')

In [ ]:
import pickle
  
# Create a variable 

  
# Open a file and use dump() 
with open('df_final.pkl', 'wb') as file: 
    # A new file will be created 
    pickle.dump(df, file) 

In [ ]:
# import marqo

# url = "http://localhost:8882"
# mq = marqo.Client(url=url)

# doc = mq.index("whole").get_document(
#     document_id="00295e5d-1860-45dd-96a9-e410e13fe233",
#     expose_facets=True
# )
# doc['_tensor_facets']

In [ ]:
# doc['_tensor_facets']

In [ ]:
# facets = doc['_tensor_facets']
# for index in range(len(doc['_tensor_facets'])):
#     print(index)
#     print(list(facets[index].keys())[0])
#     print(facets[index]['_embedding'])